In [ ]:
from google.colab import drive
import sys

#Mount your Google drive to the VM
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
sys.path.append('/content/gdrive/My Drive/Datasets/')
import os
os.chdir("/content/gdrive/Shared drives/FIT5149/Datasets/")

This file pre-processes and generates some features on labelled data, unlabelled data and test data.

# Importing libraries

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import time
import re
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('punkt')
import nltk
from nltk.tokenize import word_tokenize
from textblob import TextBlob # for extracting sentiment polarity
import string

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Reading and exploring the dataset

In [ ]:
labeled_data = pd.read_csv('labeled_data.csv')
labeled_data.shape

(50000, 2)

In [ ]:
labeled_data.head()

,text,label
0,The new rule is - \r\nif you are waiting for a...,4
1,"Flirted with giving this two stars, but that's...",3
2,I was staying at planet Hollywood across the s...,5
3,Food is good but prices are super expensive. ...,2
4,Worse company to deal with they do horrible wo...,1


In [ ]:
unlabeled_data = pd.read_csv('unlabeled_data.csv')
unlabeled_data.shape

(600000, 1)

In [ ]:
unlabeled_data.head()

,text
0,Had a good experience when my wife and I sat a...
1,On my first to Montreal with my gf we came her...
2,One of our favorite places to go when it's col...
3,"The doctor was very nice, got in in a good amo..."
4,The Nook is an immediate phoenix staple! I ca...


In [ ]:
test_data = pd.read_csv('test_data.csv')
test_data.shape

(50000, 2)

In [ ]:
test_data.head()

,test_id,text
0,test_1,trying to have a nice quiet dinner. the annou...
1,test_2,Been getting food to go from here for over 3yr...
2,test_3,Ugh. I've had to eat here a couple of times be...
3,test_4,The people here are so nice! I ordered on eat ...
4,test_5,Heard alot of good things about this place and...


## Sample Review

In [ ]:
labeled_data.text.sample(1).iloc[0]

"Second time we've been to j gumbos's and I love it! First time I tried the ratatouille and thought it was good.  But then they recommended the bumblebee stew. Awesome!! And it's vegetarian.  I've only been for dinner when it's full service, which I like. I don't think that's available during lunch. And this location has a full bar so I could enjoy a glass of wine.  Service was good both times, and David was super nice. Perfect place for a winter dinner!"

# Initial feature generation

## Defining functions for feature generation and preprocessing

In [ ]:
# for negation handling
appos = {
"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "i would",
"i'd" : "i had",
"i'll" : "i will",
"i'm" : "i am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "i have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not"
}


In [ ]:
# Returns a list of common english terms (words)
def initialize_words():
    content = None
    with open('wordlist.txt') as f: # A file containing common english words
        content = f.readlines()
    return [word.rstrip('\n') for word in content]


def parse_sentence(sentence, wordlist):
    new_sentence = "" # output    
    terms = sentence.split(' ')    
    for term in terms:
        if term[0] == '#': # this is a hashtag, parse it
            new_sentence += parse_tag(term, wordlist)
        else: # Just append the word
            new_sentence += term
        new_sentence += " "

    return new_sentence 


def parse_tag(term, wordlist):
    words = []
    # Remove hashtag, split by dash
    tags = term[1:].split('-')
    for tag in tags:
        word = find_word(tag, wordlist)    
        while word != None and len(tag) > 0:
            words.append(word)            
            if len(tag) == len(word): # Special case for when eating rest of word
                break
            tag = tag[len(word):]
            word = find_word(tag, wordlist)
    return " ".join(words)


def find_word(token, wordlist):
    i = len(token) + 1
    while i > 1:
        i -= 1
        if token[:i] in wordlist:
            return token[:i]
    return None 
  
def replaceMultiple(mainString, toBeReplaces, newString):
    for elem in toBeReplaces :
        if elem in mainString :
            mainString = mainString.replace(elem, newString)
    return  mainString

def lemmatize_text(text):
  lemmatizer = WordNetLemmatizer()
  return [lemmatizer.lemmatize(item) for item in nltk.word_tokenize(text)]
  
# for extracting some features from the data like length, word count
def some_features(data):
  data['Text Length'] = data['text'].astype(str).apply(len)
  data['Word Count'] = data['text'].apply(lambda x: len(str(x).split()))
  data['Sentiment Polarity'] = data['text'].map(lambda text: TextBlob(text).sentiment.polarity)
  data['Capital Words'] = data.text.apply(lambda x: re.findall(r'([A-Z]{2,50})',x)).str.len()
  data['Exclamation'] = data.text.apply(lambda x: re.findall(r'!',x)).str.len()
  data['Emoticons'] = data.text.apply(lambda x: re.findall(r'(?::|;|=)(?:-)?(?:\)|\(|D|P)',x)).str.len()
  data['Ques Marks'] = data.text.apply(lambda x: re.findall(r'\?',x)).str.len()
  return data


def preprocessing(data):
    # preprocessing
  data = data.drop_duplicates(keep = False)
  data = data.reset_index(drop=True)
  data = data.dropna()
  data = data.reset_index(drop=True)
  # removing urls
  data.text = data.text.apply(lambda x: re.sub(r'https?://[A-Za-z0-9./]+[-|\.|&|\+|%|=|\?*\w*]*','',x))

  # hashtag (separating hastags into word format)
  
  wordlist = initialize_words()
  temp = pd.DataFrame()
  temp['text'] = data.text
  temp['temp'] = temp.text.apply(lambda x: re.findall(r'\s*#[a-z]+',x))
  temp.temp = temp.temp.apply(lambda x: 'a' if len(x)==0 else x)
  temp.temp = temp.temp.apply(lambda x: [item.strip() for item in x])
  temp.temp = temp.temp.apply(lambda x: ' '.join(x))
  temp.temp = temp.temp.apply(lambda x: parse_sentence(x,wordlist))
  temp.temp = temp.temp.apply(lambda x: ' '+x)
  temp.text = temp.text.apply(lambda x: re.sub(r'\s*#[a-z]+','',x))
  temp.text = temp.apply(lambda x: x['text']+x['temp'],axis=1)
  data.text = temp.text

  # normalisation
  data.text = data.text.map(lambda x: x.lower())
  
  # punctuation
  punctuation_list = list(string.punctuation)
  data.text = data.text.apply(lambda x: replaceMultiple(x,punctuation_list," "))
  
  # Remove punctuations and numbers
  data.text = data.text.apply(lambda x: re.sub(r'[^a-zA-Z]',' ',x))
  # Single character removal
  data.text = data.text.apply(lambda x: re.sub(r'\s+[a-zA-Z]\s+',' ',x))
  # Removing multiple spaces
  data.text = data.text.apply(lambda x: re.sub(r'\s+',' ',x))
  
  # lemmatize text
  data.text = data.text.apply(lemmatize_text)
  
  # merging list items to create text
  data.text = data.text.apply(lambda x:" ".join(str(item) for item in x))
  
  # return clean data
  return data


## Generating features

In [ ]:
labeled_data = some_features(labeled_data)
unlabeled_data = some_features(unlabeled_data)
test_data = some_features(test_data)

In [ ]:
labeled_data.head()

,text,label,Text Length,Word Count,Sentiment Polarity,Capital Words,Exclamation,Emoticons,Ques Marks
0,The new rule is - \r\nif you are waiting for a...,4,616,111,-0.208864,1,0,0,1
1,"Flirted with giving this two stars, but that's...",3,1124,206,0.107488,0,0,0,0
2,I was staying at planet Hollywood across the s...,5,593,111,0.364722,0,1,0,0
3,Food is good but prices are super expensive. ...,2,706,130,0.030485,0,0,0,1
4,Worse company to deal with they do horrible wo...,1,630,124,-0.172321,0,0,0,0


In [ ]:
unlabeled_data.head()

,text,Text Length,Word Count,Sentiment Polarity,Capital Words,Exclamation,Emoticons,Ques Marks
0,Had a good experience when my wife and I sat a...,500,90,0.229545,0,0,0,0
1,On my first to Montreal with my gf we came her...,1624,296,0.326205,0,0,0,0
2,One of our favorite places to go when it's col...,143,27,0.300000,0,1,0,0
3,"The doctor was very nice, got in in a good amo...",707,140,0.325179,2,3,0,0
4,The Nook is an immediate phoenix staple! I ca...,1126,197,0.322321,0,1,0,0


In [ ]:
test_data.head()

,test_id,text,Text Length,Word Count,Sentiment Polarity,Capital Words,Exclamation,Emoticons,Ques Marks
0,test_1,trying to have a nice quiet dinner. the annou...,109,20,0.233333,0,0,0,0
1,test_2,Been getting food to go from here for over 3yr...,659,124,0.158929,0,0,0,0
2,test_3,Ugh. I've had to eat here a couple of times be...,1335,248,0.047309,7,3,0,0
3,test_4,The people here are so nice! I ordered on eat ...,174,32,0.337500,0,3,1,0
4,test_5,Heard alot of good things about this place and...,529,95,0.304625,0,2,0,0


# Exploratory Data Analysis

## Is the dataset balanced?

In [ ]:
labeled_data.head()

NameError: ignored

NameError: ignored

## Review Rating Distribution

In [ ]:
# !pip install plotly
import plotly.express as px

In [ ]:
fig = px.histogram(labeled_data, x="label", nbins=10)
fig.show()

NameError: ignored

## Text Length Distribution

In [ ]:
fig = px.histogram(labeled_data, x="Text Length", nbins=30)
fig.show()

## Word Count Distribution

In [ ]:
fig = px.histogram(labeled_data, x="Word Count", nbins=30)
fig.show()

## Sentiment Polarity Distribution

In [ ]:
fig = px.histogram(labeled_data, x="Sentiment Polarity", nbins=60)
fig.show()

## Sentiment Polarity grouped by Review Rating

In [ ]:
fig = px.box(labeled_data, x="label", y="Sentiment Polarity", color="label", notched= True)
fig.show()

## Review Length grouped by Review Rating

In [ ]:
fig = px.box(labeled_data, x="label", y="Text Length", color="label", notched= True)
fig.show()

## Reviews that have the lowest Polarity

In [ ]:
labeled_data[labeled_data['Sentiment Polarity'] == -1].text.head()

## Reviews that have the Lowest Ratings

In [ ]:
labeled_data[labeled_data['label'] == 1].text.head()

They all look as we expect what the negative reviews are.

## Reviews that have the highest Polarity

In [ ]:
labeled_data[labeled_data['Sentiment Polarity'] == 1].text.head()

## Reviews that have the Highest Ratings

In [ ]:
labeled_data[labeled_data['label'] == 5].text.head()

They all look as we expect what the positive reviews are.

## Reviews that have lowest polarity (most negative sentiment) but with a 5-star

In [ ]:
labeled_data[(labeled_data['label'] == 5) & (labeled_data['Sentiment Polarity'] == -1)].head(10)

## Reviews that have the highest polarity (most positive sentiment) but with a 1-star

In [ ]:
labeled_data[(labeled_data['label'] == 1) & (labeled_data['Sentiment Polarity'] == 1)].head(10)

In [ ]:
labeled_data.loc[34009, 'label'] = 5
labeled_data.loc[3429, 'Sentiment Polarity'] = -1

## Word Cloud for Positive and Negative Words

In [ ]:
# Using Nltk Sentiment Intensity Analyser to calculate Sentiment polarity of each document
# nltk.download('vader_lexicon')

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()
pos_word_list=[]
neg_word_list=[]
# Calculating the sentiment polarity of each document and then appending into positive and 
# negative word with the threshold value.
for word in  labeled_data['text']:
    if (sid.polarity_scores(word)['compound']) >= 0.8:
        pos_word_list.append(word)
    elif (sid.polarity_scores(word)['compound']) <= -0.95:
        neg_word_list.append(word)               


In [ ]:
# Generating Word Cloud using postitve words list.
# !pip install wordcloud

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt

comment_words = ' '
for words in pos_word_list: 
    comment_words = comment_words + words + ' '
  
  
wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                max_words=300,
                min_font_size = 15).generate(comment_words) 
  
# plot the WordCloud image                        
plt.figure(figsize = (9,12), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show() 

In [ ]:
#Generating Word Cloud using negative words list.
comment_words = ' '
for words in neg_word_list: 
    comment_words = comment_words + words + ' '
  
  
wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                max_words=300,
                min_font_size = 15).generate(comment_words) 
  
# plot the WordCloud image                        
plt.figure(figsize = (12,9), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show() 

# Preprocessing

In [ ]:
labeled_data = preprocessing(labeled_data)
unlabeled_data = preprocessing(unlabeled_data)
test_data = preprocessing(test_data)

In [ ]:
labeled_data.head()

,text,label,Text Length,Word Count,Sentiment Polarity,Capital Words,Exclamation,Emoticons,Ques Marks
0,the new rule is if you are waiting for table w...,4,616,111,-0.208864,1,0,0,1
1,flirted with giving this two star but that a p...,3,1124,206,0.107488,0,0,0,0
2,i wa staying at planet hollywood across the st...,5,593,111,0.364722,0,1,0,0
3,food is good but price are super expensive buc...,2,706,130,0.030485,0,0,0,1
4,worse company to deal with they do horrible wo...,1,630,124,-0.172321,0,0,0,0


In [ ]:
unlabeled_data.head()

,text,Text Length,Word Count,Sentiment Polarity,Capital Words,Exclamation,Emoticons,Ques Marks
0,had good experience when my wife and sat at th...,500,90,0.229545,0,0,0,0
1,on my first to montreal with my gf we came her...,1624,296,0.326205,0,0,0,0
2,one of our favorite place to go when it cold a...,143,27,0.300000,0,1,0,0
3,the doctor wa very nice got in in good amount ...,707,140,0.325179,2,3,0,0
4,the nook is an immediate phoenix staple came h...,1126,197,0.322321,0,1,0,0


In [ ]:
test_data.head()

,test_id,text,Text Length,Word Count,Sentiment Polarity,Capital Words,Exclamation,Emoticons,Ques Marks
0,test_1,trying to have nice quiet dinner the announcer...,109,20,0.233333,0,0,0,0
1,test_2,been getting food to go from here for over yr ...,659,124,0.158929,0,0,0,0
2,test_3,ugh ve had to eat here couple of time because ...,1335,248,0.047309,7,3,0,0
3,test_4,the people here are so nice ordered on eat and...,174,32,0.337500,0,3,1,0
4,test_5,heard alot of good thing about this place and ...,529,95,0.304625,0,2,0,0


In [ ]:
labeled_data.to_csv('preprocessed_label.csv', encoding='utf-8', index=False)
unlabeled_data.to_csv('preprocessed_unlabel.csv', encoding='utf-8', index=False)
test_data.to_csv('preprocessed_test.csv', encoding='utf-8', index=False)